In [4]:
import "dotenv/config";
import { ChatBedrockConverse } from "@langchain/aws";

const llm = new ChatBedrockConverse({
  model: "us.amazon.nova-micro-v1:0",
  region: Deno.env.BEDROCK_AWS_REGION ?? "us-east-1",
  credentials: {
    secretAccessKey: Deno.env.BEDROCK_AWS_SECRET_ACCESS_KEY ?? "",
    accessKeyId: Deno.env.BEDROCK_AWS_ACCESS_KEY_ID ?? "",
  },
  temperature: 0,
});

[Module: null prototype] { default: {}, "module.exports": {} }

## Building Blocks: The Augmented LLM

In [2]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const searchQuerySchema = z.object({
  searchQuery: z.string().describe("Query that is optimized web search."),
  justification: z.string("Why this query is relevant to the user's request."),
});

// Augment the LLM with schema for structured output
const structuredLlm = llm.withStructuredOutput(searchQuerySchema, {
  name: "searchQuery",
});

// Invoke the augmented LLM
const output = await structuredLlm.invoke(
  "How does Calcium CT score relate to high cholesterol?"
);

console.log(output);

const multiply = tool(
  async ({ a, b }) => {
    return a * b;
  },
  {
    name: "multiply",
    schema: z.object({
      a: z.number("the first number"),
      b: z.number("the second number"),
    }),
  }
);

// Augment the LLM with tools
const llmWithTools = llm.bindTools([multiply]);

// Invoke the LLM with input that triggers the tool call
const message = await llmWithTools.invoke("What is 2 times 3?");

console.log(message.tool_calls);


{
  searchQuery: "relationship between Calcium CT score and high cholesterol",
  justification: "To understand how Calcium CT score relates to high cholesterol levels."
}
[
  {
    id: "tooluse_Tzh7COltS96f2GENdodXxA",
    name: "multiply",
    args: { a: 2, b: 3 },
    type: "tool_call"
  }
]


## Prompt chaining

In [5]:
import { StateGraph, Annotation } from "@langchain/langgraph";

// Graph state
const StateAnnotation = Annotation.Root({
  topic: Annotation<string>,
  joke: Annotation<string>,
  improvedJoke: Annotation<string>,
  finalJoke: Annotation<string>,
});

// Define node functions

// First LLM call to generate initial joke
async function generateJoke(state: typeof StateAnnotation.State) {
  const msg = await llm.invoke(`Write a short joke about ${state.topic}`);
  return { joke: msg.content };
}

// Gete function to check if the joke has a puchline
function checkPunchline(state: typeof StateAnnotation.State) {
  // Simple check = does the joke contain "?" or "!"
  if (state.joke?.includes("?") || state.joke?.includes("!")) {
    return "Pass";
  }
  return "Fail";
}

// Second LLM call to improve the joke
async function improveJoke(state: typeof StateAnnotation.State) {
  const msg = await llm.invoke(
    `Make this joke funnier by adding wordplay: ${state.joke}`
  );
  return { improvedJoke: msg.content };
}

// Third LLM call for final polish
async function polishJoke(state: typeof StateAnnotation.State) {
  const msg = await llm.invoke(
    `Add a surprising twist to this joke: ${state.improvedJoke}`
  );
  return { finalJoke: msg.content };
}

// Build wordflow
const chain = new StateGraph(StateAnnotation)
  .addNode("generateJoke", generateJoke)
  .addNode("improveJoke", improveJoke)
  .addNode("polishJoke", polishJoke)
  .addEdge("__start__", "generateJoke")
  .addConditionalEdges("generateJoke", checkPunchline, {
    Pass: "improveJoke",
    Fail: "__end__",
  })
  .addEdge("improveJoke", "polishJoke")
  .addEdge("polishJoke", "__end__")
  .compile();

// Invoke
const state = await chain.invoke({ topic: "cats" });
console.log("Initial joke:");
console.log(state.joke);
console.log("\n--- --- ---\n");
if (state.improvedJoke !== undefined) {
  console.log("Improved joke:");
  console.log(state.improvedJoke);
  console.log("\n--- --- ---\n");

  console.log("Final joke:");
  console.log(state.finalJoke);
} else {
  console.log("Joke failed quality gate - no punchline detected!");
}


Initial joke:
Why did the cat go to the party alone?

Because it couldn't meow-tion anyone to come with it!

--- --- ---

Improved joke:
Why did the cat go to the party alone?

Because it couldn't meow-tion anyone to come with it—it just couldn't purr-suade anyone to join its feline-tion party!

--- --- ---

Final joke:
Why did the cat go to the party alone?

Because it couldn't meow-tion anyone to come with it—it just couldn't purr-suade anyone to join its feline-tion party! But guess what? When the party got really quiet, it found a mouse that was actually a DJ! Turns out, the cat was the only one who could really appreciate the "silent disco"!


## Parallelization

In [6]:
import { StateGraph, Annotation } from "@langchain/langgraph";

// GraphState
const StateAnnotation = Annotation.Root({
  topic: Annotation<string>,
  joke: Annotation<string>,
  story: Annotation<string>,
  poem: Annotation<string>,
  combinedOutput: Annotation<string>,
});

// Nodes
// First LLM call to generate initial joke
async function callLlm1(state: typeof StateAnnotation.State) {
  const msg = await llm.invoke(`Write a joke about ${state.topic}`);
  return { joke: msg.content };
}

// Second LLM call to generate story
async function callLlm2(state: typeof StateAnnotation.State) {
  const msg = await llm.invoke(`Write a story about ${state.topic}`);
  return { story: msg.content };
}

// Third LLM call to generate poem
async function callLlm3(state: typeof StateAnnotation.State) {
  const msg = await llm.invoke(`Write a poem about ${state.topic}`);
  return { poem: msg.content };
}

// Combine the joke, story and poem into a single output
async function aggregator(state: typeof StateAnnotation.State) {
  const combined =
    `Here's a story, joke, and poem about ${state.topic}!\n\n` +
    `STORY:\n${state.story}\n\n` +
    `JOKE:\n${state.joke}\n\n` +
    `POEM:\n${state.poem}`;
  return { combinedOutput: combined };
}

// Build workflow
const parallelWorkflow = new StateGraph(StateAnnotation)
  .addNode("callLlm1", callLlm1)
  .addNode("callLlm2", callLlm2)
  .addNode("callLlm3", callLlm3)
  .addNode("aggregator", aggregator)
  .addEdge("__start__", "callLlm1")
  .addEdge("__start__", "callLlm2")
  .addEdge("__start__", "callLlm3")
  .addEdge("callLlm1", "aggregator")
  .addEdge("callLlm2", "aggregator")
  .addEdge("callLlm3", "aggregator")
  .addEdge("aggregator", "__end__")
  .compile();

// Invoke
const result = await parallelWorkflow.invoke({ topic: "cats" });
console.log(result.combinedOutput);


Here's a story, joke, and poem about cats!

STORY:
Once upon a time in a quaint little village nestled between rolling hills and lush green meadows, there lived a curious and adventurous cat named Whiskers. Whiskers was no ordinary cat; he had a sleek, silver-gray coat that shimmered in the sunlight, and bright green eyes that sparkled with mischief. He belonged to a kind old lady named Mrs. Thompson, who lived in a cozy cottage at the edge of the village.

Whiskers was known throughout the village for his daring escapades. While other cats lounged in sunlit spots or chased after butterflies, Whiskers was always on the move, exploring every nook and cranny of the village and beyond.

One sunny morning, Whiskers decided it was time for a grand adventure. He slipped out of the cottage, his tail held high with excitement. As he trotted down the cobblestone path, he passed the village bakery, where the sweet aroma of freshly baked bread wafted through the air. Whiskers paused for a moment,

## Routing

In [5]:
import { StateGraph, Annotation } from "@langchain/langgraph";
import { z } from "zod";

// Schema for structured output to use as routing logic
const routeSchema = z.object({
  step: z
    .enum(["poem", "story", "joke"])
    .describe("The next step in the routing process"),
});

// Augment the LLM with schema for structured output
const router = llm.withStructuredOutput(routeSchema);

// Graph state
const StateAnnotation = Annotation.Root({
  input: Annotation<string>,
  decision: Annotation<string>,
  output: Annotation<string>,
});

// Nodes
// Write a story
async function llmCall1(state: typeof StateAnnotation.State) {
  const result = await llm.invoke([
    { role: "system", content: "You are an expert storyteller." },
    { role: "user", content: state.input },
  ]);
  return { output: result.content };
}

// Write a joke
async function llmCall2(state: typeof StateAnnotation.State) {
  const result = await llm.invoke([
    { role: "system", content: "You are an export comedian" },
    { role: "user", content: state.input },
  ]);
  return { output: result.content };
}

// Write a poem
async function llmCall3(state: typeof StateAnnotation.State) {
  const result = await llm.invoke([
    {
      role: "system",
      content: "You are an expert poet.",
    },
    {
      role: "user",
      content: state.input,
    },
  ]);
  return { output: result.content };
}

async function llmCallRouter(state: typeof StateAnnotation.State) {
  // Route the input to the appropriate node
  const decision = await router.invoke([
    {
      role: "system",
      content:
        "Route the input to story, joke, or poem based on the user's request.",
    },
    { role: "user", content: state.input },
  ]);
  return { decision: decision.step };
}

// Conditional edge function to route to the appropriate node
function routeDecition(state: StateAnnotation.State) {
  // Return the node name you want to visit next
  if (state.decision === "story") {
    return "llmCall1";
  } else if (state.decision === "joke") {
    return "llmCall2";
  } else if (state.decision === "poem") {
    return "llmCall3";
  }
}

// Build workflow
const routerWorkflow = new StateGraph(StateAnnotation)
  .addNode("llmCall1", llmCall1)
  .addNode("llmCall2", llmCall2)
  .addNode("llmCall3", llmCall3)
  .addNode("llmCallRouter", llmCallRouter)
  .addEdge("__start__", "llmCallRouter")
  .addConditionalEdges("llmCallRouter", routeDecition, [
    "llmCall1",
    "llmCall2",
    "llmCall3",
  ])
  .addEdge("llmCall1", "__end__")
  .addEdge("llmCall2", "__end__")
  .addEdge("llmCall3", "__end__")
  .compile();

// Invoke
const state = await routerWorkflow.invoke({
  input: "Write me a joke about cats",
});
console.log(state.output);


Sure, here's a joke for you:

Why did the cat go to the red cross?

Because it wanted to be a first-aid kit!

Hope that brought a smile to your face!


## Orchestrator-Worker

In [10]:
import { z } from "zod";
import { Annotation, StateGraph, Send } from "@langchain/langgraph";

// Schema for structured output to use in planning
const sectionSchema = z.object({
  name: z.string().describe("Name for this section of the report."),
  description: z
    .string()
    .describe(
      "Brief overview of the main topics and concepts to be covered in this section"
    ),
});

const sectionsSchema = z.object({
  sections: z.array(sectionSchema).describe("Sections of the report."),
});

// Augment the LLM with schema for structured output
const planner = llm.withStructuredOutput(sectionsSchema);

// Graph state
const StateAnnotation = Annotation.Root({
  topic: Annotation<string>,
  sections: Annotation<Array<z.infer<typeof sectionSchema>>>,
  completedSections: Annotation<string[]>({
    default: () => [],
    reducer: (a, b) => a.concat(b),
  }),
  finalReport: Annotation<string>,
});

// Worker state
const workerStateAnnotaiton = Annotation.Root({
  section: Annotation<z.infer<typeof sectionSchema>>,
  completedSections: Annotation<string[]>({
    default: () => [],
    reducer: (a, b) => a.concat(b),
  }),
});

// Nodes
async function orchestrator(state: typeof StateAnnotation.State) {
  // Generate queries
  const reportSections = await planner.invoke([
    { role: "system", content: "Generate a plan for the report." },
    { role: "user", content: `Here is the report topic: ${state.topic}` },
  ]);
  return { sections: reportSections.sections };
}

async function llmCall(state: typeof WorkerStateAnnotation.State) {
  // Generate section
  const section = await llm.invoke([
    {
      role: "system",
      content:
        "Write a report seciton following the provided name and description. Include no preamble for each section. Use markdown formatting.",
    },
    {
      role: "user",
      content: `Here is the section name: ${state.section.name} and description: ${state.section.description}`,
    },
  ]);

  // Write the updated section to completed sections
  return { completedSections: [section.content] };
}

async function synthesizer(state: typeof StateAnnotation.State) {
  // List of completed sections
  const completedSections = state.completedSections;

  // Format completed section to str to use as context for final sections
  const completedReportSections = completedSections.join("\n\n---\n\n");

  return { finalReport: completedReportSections };
}

// Conditional edge function to create llm_call workers that each write a section of the report
function assignWorkers(state: typeof StateAnnotation.State) {
  // Kick off section writing in parallel via Send() API
  return state.sections.map((section) => new Send("llmCall", { section }));
}

// Build workflow
const orchestratorWorker = new StateGraph(StateAnnotation)
  .addNode("orchestrator", orchestrator)
  .addNode("llmCall", llmCall)
  .addNode("synthesizer", synthesizer)
  .addEdge("__start__", "orchestrator")
  .addConditionalEdges("orchestrator", assignWorkers, ["llmCall"])
  .addEdge("llmCall", "synthesizer")
  .addEdge("synthesizer", "__end__")
  .compile();

// Invoke
const state = await orchestratorWorker.invoke({
  topic: "Create a report on LLM scaling laws",
});
console.log(state.finalReport);


## Introduction

This report offers a comprehensive overview of the subject matter, highlighting its significance and the rationale behind its study. The structure of the report is meticulously designed to guide the reader through the various facets of the topic in a logical and coherent manner. The sections that follow will delve into the key aspects, providing detailed insights and analysis to ensure a thorough understanding of the subject. The organization is as follows:

1. **Background Information**
2. **Importance of the Topic**
3. **Research Objectives**
4. **Methodology**
5. **Key Findings**
6. **Conclusion and Recommendations**

Each section is crafted to build upon the previous one, ensuring a seamless flow of information and a comprehensive exploration of the topic.

---

### Overview of Scaling Laws

Scaling laws in the context of large language models (LLMs) describe the relationship between the model's size, computational resources, and performance. These laws are crucial

## Evaluator-optimizer

In [5]:
import { z } from "zod";
import { Annotation, StateGraph } from "@langchain/langgraph";

// Graph state
const StateAnnotation = Annotation.Root({
  joke: Annotation<string>,
  topic: Annotation<string>,
  feedback: Annotation<string>,
  funnyOrNot: Annotation<string>,
});

// Schema for structured output to use in evaluation
const feedbackSchema = z.object({
  grade: z
    .enum(["funny", "not funny"])
    .describe("Decide if the joke is funny or not."),
  feedback: z
    .string()
    .describe(
      "If the joke is not funny, provide feedback on how to improve it."
    ),
});

// Augment the LLM with schema for structured output
const evaluator = llm.withStructuredOutput(feedbackSchema);

// Nodes
async function llmCallGenerator(state: typeof StateAnnotation.State) {
  // LLM generates a joke
  let msg;
  if (state.feedback) {
    msg = await llm.invoke(
      `Write a joke about ${state.topic} but take into account the feedback: ${state.feedback}`
    );
  } else {
    msg = await llm.invoke(`Write a joke about ${state.topic}`);
  }
  return { joke: msg.content };
}

async function llmCallEvaluator(state: typeof StateAnnotation.State) {
  // LLM evaluates the joke
  const grade = await evaluator.invoke(`Grade the joke ${state.joke}`);
  return { funnyOrNot: grade.grade, feedback: grade.feedback };
}

// Conditional edge function to route back to joke generator or end based upon feedback from the evaluator
function routeJoke(state: typeof StateAnnotation.State) {
  // Route back to joke generator or end based upon feedback from the evaluator
  if (state.funnyOrNot === "funny") {
    return "Accepted";
  } else if (state.funnyOrNot === "not funny") {
    return "Rejected + Feedback";
  }
}

// Build workflow
const optimizerWorkflow = new StateGraph(StateAnnotation)
  .addNode("llmCallGenerator", llmCallGenerator)
  .addNode("llmCallEvaluator", llmCallEvaluator)
  .addEdge("__start__", "llmCallGenerator")
  .addEdge("llmCallGenerator", "llmCallEvaluator")
  .addConditionalEdges(
    "llmCallEvaluator",
    routeJoke,
    // Name returned by routeJoke : None of next node to visit
    { Accepted: "__end__", "Rejected + Feedback": "llmCallGenerator" }
  )
  .compile();

// Invoke
const state = await optimizerWorkflow.invoke({ topic: "Cats" });
console.log(state.joke);


Sure, here's a light-hearted cat joke for you:

Why did the cat go to the party alone?

Because when it arrived, it realized it was an "mice" meeting!

Hope that brought a smile to your face!


## Agent

In [11]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

// Define tools
const multiply = tool(
  async ({ a, b }: { a: number; b: number }) => {
    return a * b;
  },
  {
    name: "multiply",
    dscription: "Multiply two numbers together",
    schema: z.object({
      a: z.number().describe("first number"),
      b: z.number().describe("second number"),
    }),
  }
);

const add = tool(
  async ({ a, b }: { a: number; b: number }) => {
    return a + b;
  },
  {
    name: "add",
    description: "Add two numbers together",
    schema: z.object({
      a: z.number().describe("first number"),
      b: z.number().describe("second number"),
    }),
  }
);

const divide = tool(
  async ({ a, b }: { a: number; b: number }) => {
    return a / b;
  },
  {
    name: "divide",
    description: "Divide two numbers",
    schema: z.object({
      a: z.number().describe("first number"),
      b: z.number().describe("second number"),
    }),
  }
);

// Augment the LLM with tools
const tools = [add, multiply, divide];
const toolsByName = Object.fromEntries(tools.map((tool) => [tool.name, tool]));
const agentWithTools = llm.bindTools(tools);

import { MessagesAnnotation, StateGraph } from "@langchain/langgraph";
import { ToolNode } from "@langchain/langgraph/prebuilt";
import { SystemMessage, ToolMessage } from "@langchain/core/messages";

// Nodes
async function agentCall(state: typeof MessagesAnnotation.State) {
  // LLM decides whether to call a tool or not
  const result = await agentWithTools.invoke([
    {
      role: "system",
      content:
        "You are a helpful assistant tasked with performing arithmetic on a set of inputs.",
    },
    ...state.messages,
  ]);

  return {
    messages: [result],
  };
}

const toolNode = new ToolNode(tools);

// Conditional edge function to route to the tool node or end
function shouldContinue(state: typeof MessagesAnnotation.State) {
  const messages = state.messages;
  const lastMessage = messages.at(-1);

  // If the LLM makes a tool call, then perform an action
  if (lastMessage?.tool_calls?.length) {
    return "Action";
  }
  // Otherwise, we stop (reply to the user)
  return "__end__";
}

// Build workflow
const agentBuilder = new StateGraph(MessagesAnnotation)
  .addNode("llmCall", agentCall)
  .addNode("tools", toolNode)
  // Add edges to connect nodes
  .addEdge("__start__", "llmCall")
  .addConditionalEdges("llmCall", shouldContinue, {
    // Name returned by shouldContinue : Name of next node to visit
    Action: "tools",
    __end__: "__end__",
  })
  .addEdge("tools", "llmCall")
  .compile();

// Invoke
const messages = [
  {
    role: "user",
    content: "Add 3 and 4.",
  },
];

const result = await agentBuilder.invoke({ messages });
console.log(result.messages);


[
  HumanMessage {
    "id": "dad0b6a0-d147-4932-a865-c3712afe08cf",
    "content": "Add 3 and 4.",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "id": "e6dc28e8-2583-40fc-af6e-2d50944f6e46",
    "content": [
      {
        "type": "text",
        "text": "<thinking> The user wants to add two numbers, 3 and 4. I will use the 'add' tool to perform this calculation.</thinking>\n"
      }
    ],
    "additional_kwargs": {},
    "response_metadata": {
      "$metadata": {
        "httpStatusCode": 200,
        "requestId": "e6dc28e8-2583-40fc-af6e-2d50944f6e46",
        "attempts": 1,
        "totalRetryDelay": 0
      },
      "metrics": {
        "latencyMs": 874
      },
      "stopReason": "tool_use",
      "usage": {
        "inputTokens": 621,
        "outputTokens": 51,
        "totalTokens": 672
      }
    },
    "tool_calls": [
      {
        "id": "tooluse_6TS9Vf6_Sg-SLlg_DU-4VQ",
        "name": "add",
        "args": {
          "a": 3,
    

### Pre-build

In [13]:
import { createReactAgent } from "@langchain/langgraph/prebuilt";

// Pass in:
// (1) an LLM instance
// (2) the tools list (which is used to create the tool node)
const prebuiltAgent = createReactAgent({
  llm: agentWithTools,
  tools,
});

// invoke
const result = await prebuiltAgent.invoke({
  messages: [
    {
      role: "user",
      content: "Multiply 3 and 4.",
    },
  ],
});
console.log(result.messages);


[
  HumanMessage {
    "id": "21dde8df-4601-4c48-a060-cefae7fd5dd0",
    "content": "Multiply 3 and 4.",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "id": "1e13ec15-2aaa-4266-babe-cc7d338b425e",
    "content": [
      {
        "type": "text",
        "text": "<thinking> The user wants to multiply two numbers: 3 and 4. I can use the \"multiply\" tool to perform this operation. </thinking>\n"
      }
    ],
    "additional_kwargs": {},
    "response_metadata": {
      "$metadata": {
        "httpStatusCode": 200,
        "requestId": "1e13ec15-2aaa-4266-babe-cc7d338b425e",
        "attempts": 1,
        "totalRetryDelay": 0
      },
      "metrics": {
        "latencyMs": 751
      },
      "stopReason": "tool_use",
      "usage": {
        "inputTokens": 607,
        "outputTokens": 49,
        "totalTokens": 656
      }
    },
    "tool_calls": [
      {
        "id": "tooluse_65wj985GSwy4iiBYWGO5JQ",
        "name": "multiply",
        "args": {
  